In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
cd /raid/home/myang349/recsys-filterbubbles/

/raid/home/myang349/recsys-filterbubbles


# Imports

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.optim as optim
from os import listdir
from os.path import isfile, join
from tracin.tracin import (
    save_tracin_checkpoint,
    load_tracin_checkpoint,
    calculate_tracin_influence,
)
import pandas as pd
from LSTM_clean.utils import filter_and_split_data, sequence_generator, load_community_dict, get_communities 
from LSTM_clean.model import LSTM 
from collections import Counter
import numpy as np
import pickle
from collections import defaultdict
import copy
from torch.optim import SGD
from lstm_wrapper import train_model, get_topk_predictions
from dataclasses import dataclass

# Loading Data

In [6]:
# Data Location
# Use SAVE_PREFIX to mark different datasets
SAVE_PREFIX = "100k"
SAVE_FOLDER = "/raid/home/myang349/recsys-filterbubbles/data/twitch_sequence/"
SAVE_TRAIN_NAME = SAVE_PREFIX + "train.data"
SAVE_VALID_NAME = SAVE_PREFIX + "valid.data"
SAVE_TEST_NAME = SAVE_PREFIX + "test.data"
SAVE_COMMUNITY_NAME = SAVE_PREFIX + "lstm_idx_communities.data"

In [7]:
train_data = np.load(os.path.join(SAVE_FOLDER, SAVE_TRAIN_NAME), allow_pickle=True)
valid_data = np.load(os.path.join(SAVE_FOLDER, SAVE_VALID_NAME), allow_pickle=True)
test_data = np.load(os.path.join(SAVE_FOLDER, SAVE_TEST_NAME), allow_pickle=True)

# Training

In [8]:
model = train_model()

Device is cuda
Train: 71465, Valid: 8913
Model is  LSTM(
  (item_emb): Embedding(5400, 128, padding_idx=0)
  (LSTM): LSTM(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=5400, bias=True)
)

Training and testing
train # = 71465, test # = 8913

Epoch 0	Train Loss: 0.016824991931727123	Test MRR: 0.0033817594200573073	Test Recall@10: 0.005834174800852687	Elapsed time: 1.9552690982818604
saving checkpoint to /raid/home/myang349/recsys-filterbubbles/checkpoints/lstm_checkpoint_epoch0.pt
Epoch 5	Train Loss: 0.0166483138261962	Test MRR: 0.03344252038281658	Test Recall@10: 0.06181981375518905	Elapsed time: 7.962569952011108
Epoch 10	Train Loss: 0.01630291481394385	Test MRR: 0.04028087609152543	Test Recall@10: 0.08661505665881297	Elapsed time: 8.038089513778687
saving checkpoint to /raid/home/myang349/recsys-filterbubbles/checkpoints/lstm_checkpoint_epoch10.pt
Epoch 15	Train Loss: 0.015593860768741401	Test MRR: 0.04103156270250585	Test Recall@10: 0.08179064288118479	Elapse

# Experimentation

## Verify the recall

In [9]:
# Generate sequences and gt
sequences = [pt[0] for pt in valid_data]
gt = [pt[1] for pt in valid_data]

In [10]:
# Generate predictions
predictions = get_topk_predictions(model, sequences, k=50)

In [11]:
assert len(sequences) == len(predictions)
hit = 0
total = 0
for i, val in enumerate(gt):
    if val in predictions[i]:
        hit += 1
    total += 1  
hit/total

0.6748569505217099

## Diversity of watch history vs diversity of predictions

In [12]:
community_dict = load_community_dict(os.path.join(SAVE_FOLDER, SAVE_COMMUNITY_NAME))

In [13]:
# Converting prev and predictions to be equal length and in community embedding
prev_comm = []
pred_comm = []
for i in range(len(sequences)):
    prev = sequences[i]
    n = len([x for x in prev if x != 0])
    pred = predictions[i][:n]
    
    prev_comm.append(get_communities(prev, community_dict))
    pred_comm.append(get_communities(pred, community_dict))

In [14]:
for i in range(5):
    assert len(prev_comm[i]) ==len(pred_comm[i])

In [20]:
prev_num_comm = 0
pred_num_comm = 0
n = len(prev_comm)
threshold = 0

for i in range(n):
    if len(prev_comm[i]) < threshold:
        continue
    prev_num_comm += len(set(prev_comm[i]))
    pred_num_comm += len(set(pred_comm[i]))

In [21]:
prev_num_comm

21446

In [22]:
pred_num_comm

33416

# Analyzing recommendation distribution

In [103]:
min(len(x) for x in pred_comm)

8

In [105]:
prev_num_comm

15550

In [102]:
pred_num_comm

28544

In [106]:
pred_comm[0]

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]